In [8]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

In [545]:
def gt_fn(x):
    #print x
    x = x.reshape(-1,4)
    return (np.max(x[:,[0,1]], axis=1) > ((x[:,2]) + x[:,3]/2.0))*1.0

In [546]:
gt_fn(np.random.rand(2,1,4))

array([ 1.,  0.])

In [549]:
X_train = np.random.rand(100000,4)*10
y_train = gt_fn(X_train)

X_test = np.random.rand(10000,4)*10
y_test = gt_fn(X_test)

In [548]:
import sklearn
def sample(X_batch, y_batch, size, num_classes):
    
    X_batch = np.asarray(X_batch)
    y_batch = np.asarray(y_batch)
    
    x_data = []
    y_data = []
    
    for i in range(num_classes):
        x_data.append(sklearn.utils.resample(X_batch[np.where(y_batch == i)[0]], n_samples=size))
        y_data.append(np.zeros((size, 1))+i)


    X_batch = np.concatenate(x_data)
    y_batch = np.concatenate(y_data)

    todo = range(len(y_batch))
    np.random.shuffle(todo)

    X_batch = X_batch[todo]
    y_batch = y_batch[todo]
    
    return X_batch, y_batch

#sample(X_train, y_train, 10,2)

In [114]:
edges_np = np.array([(0,3),(3,1),(1,2),(2,3)])
edges_np = np.concatenate([edges_np,edges_np[:,[1,0]]])

In [115]:
edges_np

array([[0, 3],
       [3, 1],
       [1, 2],
       [2, 3],
       [3, 0],
       [1, 3],
       [2, 1],
       [3, 2]])

In [13]:
edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(4)])

In [14]:
edge_selector

array([array([0]), array([2, 5]), array([3, 6]), array([1, 4, 7])], dtype=object)

In [59]:
edges = torch.LongTensor(edges_np)

In [60]:
edges


    0     3
    3     1
    1     2
    2     3
    3     0
    1     3
    2     1
    3     2
[torch.LongTensor of size 8x2]

In [71]:
weights = Variable(torch.rand(edges.shape), requires_grad=True)

In [72]:
weights

Variable containing:
 0.5684  0.1352
 0.0188  0.3241
 0.6176  0.1497
 0.6121  0.2223
 0.6169  0.3865
 0.9437  0.9026
 0.6818  0.4499
 0.3595  0.6131
[torch.FloatTensor of size 8x2]

In [19]:
# tocompute = torch.index_select(x, 1, Variable(edges.view(-1))).view(2,-1, 2)
# tocompute

In [694]:
class CGN2(nn.Module):
    def __init__(self,input_dim, edges, channels=1, out_dim=2, on_cuda=True):
        super(CGN2, self).__init__()

        self.my_layers = []
        self.out_dim = out_dim
        self.on_cuda = on_cuda
        self.edges = edges
        self.channels = channels
        #dims = [input_dim] + channels
        
        print "Constructing the network..."   
        
        
        self.weights1 = nn.Parameter(torch.rand(edges.shape), requires_grad=True)
        print self.weights1.size()

        self.last_layer = nn.Linear(input_dim, out_dim)
        self.my_layers = nn.ModuleList([self.last_layer])

        print "Done!"

        
    #batch_size = 2
    #print x
    def GraphConv(self, x, edges, batch_size, weights):
        
        x = x.clone()
        x = x.view(batch_size, -1)
        
        tocompute = torch.index_select(x, 1, Variable(edges.view(-1))).view(batch_size, -1, 2)
        #print tocompute
        conv = tocompute*weights
        #print conv
        for i, edges_to_select in enumerate(edge_selector):
            #print "x", conv
            #print "e", edges_to_select
            selected_edges = torch.index_select(conv, 1, Variable(torch.LongTensor(edges_to_select)))
            #print "m", selected_edges
            selected_edges = selected_edges.view(-1,edges_to_select.shape[0]*2)
            #print "m", selected_edges
            pooled_edges = torch.max(selected_edges,1)[0]
            #print "mmo",pooled_edges
            x[:,i] = pooled_edges
            #print "xx",x[:,i]
        return x

        
        
    def forward(self, x):
        nb_examples, nb_nodes, nb_channels = x.size()
        
        self.GraphConv(x,self.edges, nb_examples, self.weights1)

        x = self.last_layer(x.view(nb_examples, -1))
        x = F.softmax(x)

        return x

In [695]:
import models
reload(models);

In [696]:
gc = CGN2(4,edges)
#gc = models.CGN2(4,edges)
#gc = models.MLP(4,[10],2)
gc

Constructing the network...
torch.Size([8, 2])
Done!


CGN2 (
  (last_layer): Linear (4 -> 2)
  (my_layers): ModuleList (
    (0): Linear (4 -> 2)
  )
)

In [697]:
list(gc.parameters())

[Parameter containing:
  0.9015  0.2536
  0.3299  0.4461
  0.8892  0.1046
  0.5125  0.3485
  0.5106  0.7401
  0.3868  0.6805
  0.5301  0.6224
  0.9447  0.7105
 [torch.FloatTensor of size 8x2], Parameter containing:
  0.3929 -0.2951  0.1771 -0.1583
  0.1390  0.1762  0.0484  0.3792
 [torch.FloatTensor of size 2x4], Parameter containing:
 -0.2273
  0.0437
 [torch.FloatTensor of size 2]]

In [698]:
optimizer = optim.SGD(gc.parameters(), lr = 0.001, momentum=0.9)

In [710]:
X_batch, y_batch = sample(X_train, y_train, 3,2)

In [741]:
x = Variable(torch.FloatTensor(X_batch)).view(-1,1,4)
x

Variable containing:
(0 ,.,.) = 
  9.2859  0.4024  6.1553  0.7816

(1 ,.,.) = 
  5.8605  6.1283  5.6909  0.3942

(2 ,.,.) = 
  3.3339  8.4306  6.8196  3.6241

(3 ,.,.) = 
  6.0164  8.3235  9.4050  0.0272

(4 ,.,.) = 
  5.9386  7.4645  1.9813  4.2063

(5 ,.,.) = 
  5.8822  9.9457  8.9656  5.8690
[torch.FloatTensor of size 6x1x4]

In [742]:
output = gc(x)[:,1]

In [743]:
output

Variable containing:
 0.1213
 0.8014
 0.9909
 0.8533
 0.9894
 0.9969
[torch.FloatTensor of size 6]

In [744]:
gt = Variable(torch.FloatTensor(y_batch), requires_grad=False).view(-1)
loss = torch.abs(gt-output)
print loss
print loss.mean()

Variable containing:
 0.8787
 0.1986
 0.9909
 0.8533
 0.0106
 0.9969
[torch.FloatTensor of size 6]

Variable containing:
 0.6549
[torch.FloatTensor of size 1]



In [745]:
loss.mean().backward()
optimizer.step()

In [705]:
X_batch.shape

(2, 4)

In [706]:
x = Variable(torch.FloatTensor(X_batch))
output = gc(x)
#gt = Variable(torch.FloatTensor(gt_fn(x.data.numpy()))).view(batch_size,-1)
#loss = torch.abs(gt-output)
#loss

ValueError: need more than 2 values to unpack

In [707]:
y_batch

array([[ 1.],
       [ 0.]])

In [57]:
torch.manual_seed(0)
x = Variable(torch.rand(batch_size,4)*10)
output = torch.nn.Sigmoid()(linearLayer(x)).view(-1)
loss = torch.abs(Variable(gt_fn(x.data))-output)
print  loss

Variable containing:
 0.1381
 0.1273
[torch.FloatTensor of size 2]



In [58]:
loss.mean().backward()
optimizer.step()